In [5]:
import os
import sys
import numpy as np
import scipy as sc
import pandas as pd
import seaborn as sns
import matplotlib.pyplot as plt
import warnings
warnings.filterwarnings("ignore")

In [6]:
from pyspark.sql import SparkSession, Row
from pyspark.sql.functions import *
from pyspark.sql.types import *

In [7]:
MAX_MEMORY = "10g"

spark = SparkSession \
    .builder \
    .appName("Foo") \
    .config("spark.executor.memory", MAX_MEMORY) \
    .config("spark.driver.memory", MAX_MEMORY) \
    .getOrCreate()

In [8]:
user = 540
book = 26

In [9]:
print(spark.catalog.listDatabases())

[Database(name='default', description='Default Hive database', locationUri='file:/home/big/Desktop/Aulas/Projeto/spark-warehouse')]


In [10]:
spark.catalog.listTables(dbName="default")

[Table(name='itemsrecommendationtable', database='default', description=None, tableType='MANAGED', isTemporary=False),
 Table(name='itemstable', database='default', description=None, tableType='MANAGED', isTemporary=False),
 Table(name='userrecommendationtable', database='default', description=None, tableType='MANAGED', isTemporary=False)]

In [11]:
spark.sql("USE default")

DataFrame[]

In [12]:
spark.catalog.listColumns("itemstable")

[Column(name='_c2', description=None, dataType='double', nullable=True, isPartition=False, isBucket=False),
 Column(name='_c3', description=None, dataType='int', nullable=True, isPartition=False, isBucket=False),
 Column(name='Item_Index', description=None, dataType='double', nullable=True, isPartition=False, isBucket=False),
 Column(name='User_Index', description=None, dataType='double', nullable=True, isPartition=False, isBucket=False)]

In [13]:
spark.sql("SELECT * FROM itemstable").show(10,truncate=False)

+---+----------+----------+----------+
|_c2|_c3       |Item_Index|User_Index|
+---+----------+----------+----------+
|5.0|1362268800|3562.0    |9001.0    |
|4.0|1389398400|12294.0   |1960.0    |
|4.0|1362960000|1132.0    |185.0     |
|5.0|1405468800|14353.0   |9987.0    |
|5.0|1398470400|6896.0    |6297.0    |
|5.0|1271116800|2943.0    |3256.0    |
|5.0|1363392000|813.0     |14429.0   |
|5.0|1380067200|2351.0    |11613.0   |
|3.0|1385683200|320.0     |3944.0    |
|3.0|1246406400|6591.0    |1345.0    |
+---+----------+----------+----------+
only showing top 10 rows



In [14]:
spark.sql("SELECT * FROM userrecommendationtable").show(10,truncate=False)

+----------+-----------------------------------------------------------------------------------------+
|User_Index|recommendations                                                                          |
+----------+-----------------------------------------------------------------------------------------+
|35        |[[56, 6.3561664], [113, 5.561143], [5566, 4.997807], [274, 4.8611126], [256, 4.5964136]] |
|1815      |[[279, 6.3834496], [27, 6.0663486], [553, 4.997807], [81, 4.7624025], [64, 4.7132783]]   |
|2445      |[[2, 6.369457], [394, 6.2148685], [109, 5.0039024], [75, 4.802309], [325, 4.7997465]]    |
|2915      |[[74, 5.0404563], [415, 4.4797378], [81, 4.471573], [205, 4.4445376], [10518, 4.4242463]]|
|3065      |[[1, 9.549492], [33, 6.168268], [49, 6.001958], [41, 5.056066], [131, 4.9294915]]        |
|3545      |[[83, 5.898951], [383, 5.130368], [12777, 4.9978075], [82, 4.939535], [48, 4.775062]]    |
|3875      |[[207, 5.752092], [61, 5.512059], [15, 5.2211194], [15244, 4.

In [15]:
spark.sql("SELECT * FROM itemsrecommendationtable").show(10,truncate=False)

+----------+----------------------------------------------------------------------------------------------------+
|Item_Index|recommendations                                                                                     |
+----------+----------------------------------------------------------------------------------------------------+
|35        |[[5401, 6.4186707], [13133, 6.3704214], [4038, 6.345374], [3754, 6.12169], [5616, 5.9706116]]       |
|503       |[[2620, 4.997807], [15662, 4.546013], [8390, 4.443827], [14271, 4.3867855], [4499, 4.1742606]]      |
|583       |[[6732, 0.9899167], [3782, 0.91787565], [10121, 0.9102786], [13268, 0.9035039], [11325, 0.89800096]]|
|610       |[[707, 4.997807], [1182, 4.5763116], [11624, 4.378967], [13517, 4.3635087], [3669, 4.2734566]]      |
|614       |[[9788, 4.997807], [13850, 4.4660597], [1177, 4.3294864], [478, 4.244074], [14427, 4.234099]]       |
|761       |[[6279, 4.997807], [4632, 4.493599], [13110, 4.3835387], [4777, 4.338821], [